# Preprocesamiento

In [52]:
import pandas as pd

import sys
sys.path.append("..")

from src.support_prep import *
from src.support_models import *    

import pickle

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
df = pd.read_pickle("../datos/clean.pkl")

In [55]:
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,...,1,6,1,0,0,3.0,4,2,3,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,2,Female,1,...,6,3,5,1,4,3.0,2,4,2,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,3,Male,4,...,5,2,5,0,3,2.0,2,1,3,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,4,Male,3,...,13,5,8,7,5,4.0,4,3,2,3
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,5,Male,1,...,9,2,6,0,4,4.0,1,3,3,3


## Encoding

Para realizar el encoding, vamos a necesitar convertir nuestra variable respuesta en binario.

In [56]:
df["Attrition"] = df["Attrition"].map({"No":0, "Yes":1})

In [57]:
cat_cols = df.select_dtypes("O").columns
num_cols = df.select_dtypes("number").columns.drop(["Attrition", "EmployeeID"])

In [58]:
df = df.drop(columns="EmployeeID")

In [59]:
cat_diff = detectar_orden_cat(df = df, lista_cat=cat_cols,var_respuesta="Attrition", show=False)

In [60]:
cat_no_diff = cat_cols.drop(cat_diff)

In [61]:
df_target, target = encode_target(data = df, columns = cat_diff, response_var="Attrition")

In [62]:
df_oh, onehot = encode_onehot(df, columns = cat_no_diff)

In [63]:
df = pd.concat([df["Attrition"], df_target.drop(columns=cat_no_diff), df_oh], axis = 1)

In [64]:
with open('../models/onehot.pkl', 'wb') as file:
    pickle.dump(onehot, file)
with open('../models/target.pkl', 'wb') as file:
    pickle.dump(target, file)

## Feature Scaling

In [65]:
df_scaled, scaler = scale_data(data = df, columns=df.columns.drop(df_oh.columns).drop("Attrition"), method = "minmax")

In [66]:
df[df_scaled.columns] = df_scaled

In [67]:
with open('../models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## Outliers

In [68]:
df_outliers, ifo = find_outliers(data = df.drop(columns = "Attrition"), columns = df.drop(columns = "Attrition").columns, method = "ifo")

100%|██████████| 25/25 [00:47<00:00,  1.90s/it]


In [69]:
df.drop(index=df_outliers.index, inplace=True)

In [70]:
df.reset_index(drop=True, inplace=True)

In [71]:
df_outliers, lof = find_outliers(data = df.drop(columns = "Attrition"), columns = df.drop(columns = "Attrition").columns, method = "lof")

100%|██████████| 25/25 [00:02<00:00, 11.59it/s]


In [72]:
df.loc[df_outliers.index, num_cols] = np.nan

In [73]:
df_out_impute, out_imputer = impute_nulls(df)

[IterativeImputer] Completing matrix with shape (4321, 38)
[IterativeImputer] Ending imputation round 1/10, elapsed time 16.18
[IterativeImputer] Change: 1.0291480284380448, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 2/10, elapsed time 34.18
[IterativeImputer] Change: 0.7987345141848214, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 3/10, elapsed time 54.12
[IterativeImputer] Change: 0.566579591868343, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 4/10, elapsed time 75.29
[IterativeImputer] Change: 0.5860132599643331, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 5/10, elapsed time 97.68
[IterativeImputer] Change: 0.6162944185573119, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 6/10, elapsed time 121.51
[IterativeImputer] Change: 0.6951865079365078, scaled tolerance: 0.0010000000000000002 
[Iter

In [74]:
df = df_out_impute

In [75]:
df.to_pickle("../datos/prepped.pkl")